In [14]:
import json
import numpy as np
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
kb = json.loads(open('src/experiments/kb.txt').readline())

kb_source = kb['yeast']
kb_target = kb['twitter']

In [16]:
source = 'proteinclass'
target = 'accounttype'
             

In [17]:
pred_target = create_pred_target(kb_target)

In [18]:
yeast_dataset = datasets.load('yeast', kb_source, target=source, seed=441773, balanced=0) #facts, pos, neg
twitter_dataset = datasets.load('twitter', kb_target, target=target, seed=441773, balanced=0) #facts, pos, neg

# train_facts_source, train_pos_source, train_neg_source = get_train_division(yeast_dataset)
# background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
#                                       maxTreeDepth=3, nodeSize=2, numOfClauses=8)
# model_src = boostsrl.train(background_knowledge_src, train_pos_source, np.random.choice(train_neg_source, 2*len(train_pos_source)), train_facts_source, trees=10)

# structured_src, new_src_struct = create_structured_trees(model_src)

In [6]:
ss = [['proteinclass(A, B)',
  {'': 'interaction(C, A), recursion_proteinclass(C, B)',
   'true': 'interaction(D, C), recursion_proteinclass(D, B)',
   'true,false': 'interaction(A, E), interaction(E, E)',
   'false': 'interaction(A, F), recursion_proteinclass(F, B)',
   'false,true': 'interaction(F, G), phenotype(G, H)',
   'false,true,true': 'interaction(G, I), recursion_proteinclass(I, B)',
   'false,false': 'interaction(A, A)',
   'false,false,false': 'location(A, J), complex(A, K)'},
  {'true,true': [2.6e-07, 0, 37],
   'true,false,true': [0.816, 2, 1],
   'true,false,false': [0.294, 2, 19],
   'false,true,true,true': [1.095, 3, 2],
   'false,true,true,false': [2.98e-08, 0, 7],
   'false,true,false': [0.0, 0, 14],
   'false,false,true': [5.58e-08, 53, 0],
   'false,false,false,true': [8.562, 291, 98],
   'false,false,false,false': [11.309, 387, 191]}],
 ['proteinclass(A, B)',
  {'': 'interaction(C, A), recursion_proteinclass(C, B)',
   'false': 'interaction(D, A), interaction(A, D)',
   'false,true': 'interaction(D, D)',
   'false,true,true': 'interaction(A, A)',
   'false,true,false': 'enzyme(D, E), complex(A, F)',
   'false,false': 'location(A, G), complex(A, H)',
   'false,false,true': 'complex(I, H), recursion_proteinclass(I, B)',
   'false,false,false': 'interaction(J, A)'},
  {'true': [0.234, 4, 57],
   'false,true,true,true': [0.0, 53, 0],
   'false,true,true,false': [3.024, 31, 13],
   'false,true,false,true': [0.949, 9, 1],
   'false,true,false,false': [4.41, 71, 28],
   'false,false,true,true': [0.355, 0, 91],
   'false,false,true,false': [0.02, 232, 0],
   'false,false,false,true': [10.008, 301, 152],
   'false,false,false,false': [3.922, 37, 27]}],
 ['proteinclass(A, B)',
  {'': 'interaction(C, A), recursion_proteinclass(C, B)',
   'true': 'interaction(D, C), recursion_proteinclass(D, B)',
   'false': 'interaction(E, A), interaction(A, F)',
   'false,true': 'recursion_proteinclass(F, B)',
   'false,true,false': 'interaction(E, F), enzyme(F, G)',
   'false,false': 'interaction(A, H), recursion_proteinclass(H, B)',
   'false,false,true': 'interaction(H, I), phenotype(H, J)',
   'false,false,false': 'complex(A, K)'},
  {'true,true': [0.0, 0, 37],
   'true,false': [0.353, 4, 20],
   'false,true,true': [0.209, 1, 16],
   'false,true,false,true': [3.489, 97, 15],
   'false,true,false,false': [7.356, 230, 80],
   'false,false,true,true': [0.584, 2, 1],
   'false,false,true,false': [0.16, 0, 6],
   'false,false,false,true': [5.506, 153, 60],
   'false,false,false,false': [9.342, 251, 134]}],
 ['proteinclass(A, B)',
  {'': 'interaction(A, C), enzyme(C, D)',
   'true': 'interaction(E, A), recursion_proteinclass(E, B)',
   'true,false': 'interaction(A, A)',
   'true,false,false': 'phenotype(C, F), complex(C, G)',
   'false': 'location(A, H), complex(A, I)',
   'false,false': 'enzyme(A, J)',
   'false,false,true': 'enzyme(K, J), recursion_proteinclass(K, B)',
   'false,false,false': 'interaction(A, L), phenotype(L, M)'},
  {'true,true': [0.23, 2, 22],
   'true,false,true': [0.0, 33, 0],
   'true,false,false,true': [3.889, 57, 28],
   'true,false,false,false': [4.469, 105, 34],
   'false,true': [0.375, 217, 101],
   'false,false,true,true': [0.428, 0, 60],
   'false,false,true,false': [0.311, 91, 0],
   'false,false,false,true': [3.901, 46, 28],
   'false,false,false,false': [7.753, 187, 96]}],
 ['proteinclass(A, B)',
  {'': 'interaction(A, C), enzyme(C, D)',
   'true': 'recursion_proteinclass(C, B)',
   'false': 'complex(A, E), location(A, F)',
   'false,true': 'location(G, F), recursion_proteinclass(G, B)',
   'false,true,true': 'complex(G, E)',
   'false,false': 'interaction(A, H), enzyme(A, I)',
   'false,false,true': 'interaction(H, J), recursion_proteinclass(J, B)',
   'false,false,false': 'phenotype(A, K), interaction(L, A)'},
  {'true,true': [0.141, 1, 17],
   'true,false': [0.373, 196, 67],
   'false,true,true,true': [1.198, 0, 101],
   'false,true,true,false': [0.389, 50, 0],
   'false,true,false': [0.199, 167, 0],
   'false,false,true,true': [0.75, 1, 6],
   'false,false,true,false': [2.555, 27, 16],
   'false,false,false,true': [6.012, 130, 64],
   'false,false,false,false': [7.36, 166, 98]}],
 ['proteinclass(A, B)',
  {'': 'interaction(A, C), recursion_proteinclass(C, B)',
   'false': 'interaction(A, D), interaction(E, A)',
   'false,true': 'interaction(E, D), interaction(F, E)',
   'false,true,true': 'interaction(F, D), recursion_proteinclass(F, B)',
   'false,true,false': 'complex(A, G), interaction(D, H)',
   'false,false': 'complex(A, I)',
   'false,false,true': 'complex(J, I), recursion_proteinclass(J, B)',
   'false,false,false': 'interaction(A, K), complex(K, L)'},
  {'true': [0.187, 4, 57],
   'false,true,true,true': [1.547, 2, 27],
   'false,true,true,false': [3.434, 202, 21],
   'false,true,false,true': [2.395, 44, 12],
   'false,true,false,false': [4.765, 80, 42],
   'false,false,true,true': [0.784, 0, 69],
   'false,false,true,false': [0.368, 154, 0],
   'false,false,false,true': [2.756, 20, 16],
   'false,false,false,false': [8.409, 232, 125]}],
 ['proteinclass(A, B)',
  {'': 'location(A, C), complex(A, D)',
   'true': 'location(E, C), recursion_proteinclass(E, B)',
   'true,true': 'complex(E, D)',
   'false': 'interaction(A, F), recursion_proteinclass(F, B)',
   'false,true': 'interaction(F, G), enzyme(G, H)',
   'false,false': 'interaction(I, A), recursion_proteinclass(I, B)',
   'false,false,true': 'complex(I, J)',
   'false,false,false': 'interaction(A, K), interaction(K, A)'},
  {'true,true,true': [0.146, 0, 146],
   'true,true,false': [0.076, 85, 0],
   'true,false': [0.365, 236, 0],
   'false,true,true': [0.26, 2, 10],
   'false,true,false': [0.056, 0, 13],
   'false,false,true,true': [0.668, 1, 2],
   'false,false,true,false': [0.173, 0, 7],
   'false,false,false,true': [3.238, 77, 20],
   'false,false,false,false': [10.012, 337, 171]}],
 ['proteinclass(A, B)',
  {'': 'interaction(A, C), recursion_proteinclass(C, B)',
   'true': 'interaction(C, D), complex(D, E)',
   'false': 'location(A, F), complex(A, G)',
   'false,true': 'location(H, F), recursion_proteinclass(H, B)',
   'false,true,true': 'complex(H, G)',
   'false,false': 'interaction(A, I), interaction(I, A)',
   'false,false,true': 'interaction(A, A)',
   'false,false,false': 'interaction(A, J), complex(J, K)'},
  {'true,true': [0.19, 4, 34],
   'true,false': [0.067, 0, 23],
   'false,true,true,true': [1.357, 0, 112],
   'false,true,true,false': [0.62, 84, 0],
   'false,true,false': [0.297, 235, 0],
   'false,false,true,true': [0.105, 27, 0],
   'false,false,true,false': [3.17, 50, 20],
   'false,false,false,true': [4.009, 46, 32],
   'false,false,false,false': [9.192, 292, 148]}],
 ['proteinclass(A, B)',
  {'': 'interaction(A, C), recursion_proteinclass(C, B)',
   'true': 'interaction(C, D), complex(D, E)',
   'false': 'complex(A, F), location(A, G)',
   'false,true': 'location(H, G), recursion_proteinclass(H, B)',
   'false,true,true': 'complex(H, F)',
   'false,false': 'interaction(A, I), interaction(I, I)',
   'false,false,true': 'interaction(A, A)',
   'false,false,false': 'interaction(J, A), interaction(J, J)'},
  {'true,true': [0.192, 4, 34],
   'true,false': [0.059, 0, 23],
   'false,true,true,true': [1.233, 0, 112],
   'false,true,true,false': [0.588, 84, 0],
   'false,true,false': [0.271, 235, 0],
   'false,false,true,true': [0.096, 27, 0],
   'false,false,true,false': [3.394, 44, 21],
   'false,false,false,true': [3.76, 45, 29],
   'false,false,false,false': [9.206, 299, 150]}],
 ['proteinclass(A, B)',
  {'': 'complex(A, C), location(A, D)',
   'false': 'interaction(A, E), enzyme(E, F)',
   'false,true': 'complex(E, G), interaction(E, H)',
   'false,true,true': 'phenotype(E, I), location(A, J)',
   'false,true,false': 'interaction(E, K), recursion_proteinclass(K, B)',
   'false,false': 'interaction(A, L), phenotype(L, M)',
   'false,false,true': 'phenotype(A, M), interaction(L, A)',
   'false,false,false': 'enzyme(A, N), location(A, P)'},
  {'true': [0.162, 321, 146],
   'false,true,true,true': [1.448, 4, 6],
   'false,true,true,false': [1.434, 6, 4],
   'false,true,false,true': [1.338, 1, 13],
   'false,true,false,false': [3.076, 82, 16],
   'false,false,true,true': [0.839, 10, 3],
   'false,false,true,false': [4.174, 54, 39],
   'false,false,false,true': [3.892, 66, 42],
   'false,false,false,false': [7.755, 194, 100]}]]

In [7]:
import copy
src_struct = copy.deepcopy(ss)
new_src_struct = []
for i in range(0, len(src_struct)):
    new_src_struct.append(define_individual(src_struct[i], i))  
structured_src = src_struct


In [8]:
twitter_dataset = datasets.load('twitter', kb_target, target=target, balanced=False) #facts, pos, neg
train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(twitter_dataset, 1)

# train_facts, train_pos, train_neg = twitter_dataset[0], twitter_dataset[1], twitter_dataset[2]
tt = []
tt.extend(train_facts[0])
tt.extend(train_facts[1])
tt.extend(test_facts)
train_facts = [list(set(tt)), list(set(tt))]

1


# Crossover original

## Sem revisão

In [13]:
start = time.time()
res_c_r = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=10)
print("FIM DO GROOT GENETIC: ", time.time()-start)

Process ForkPoolWorker-106:
Process ForkPoolWorker-105:
Process ForkPoolWorker-104:
Process ForkPoolWorker-107:
Process ForkPoolWorker-100:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/multiprocessing/process.py", line 93, in run
  